Default URL to display dashboard:
http://127.0.0.1:8050/

# Dashboard

In [ ]:
# API imports
import requests
from bson.objectid import ObjectId
from bson.son import SON

# dashboard components
from dash import Dash, dcc, html, dash_table
app = Dash(__name__)

import plotly.express as px
from dash.dependencies import Input, Output, State
import base64

# OS routines
import os

# plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from CR_py import CRUD_DB # import py file for MongoDB access
#---------------------------------------------------------------------------------------------#
###########################
# Data Manipulation / Model
###########################

# CDC API
dataset_url = "https://data.cdc.gov/resource/r8kw-7aab.json"
response = requests.get(dataset_url)
data = response.json()
df = pd.DataFrame.from_dict(data)

# Connect to MongoDB
db = CRUD_DB()
db.create_db(df)
documents = db.read({})  # Read the documents
df_db = pd.DataFrame(documents)  # Fetch all documents

# Debugging output
print(len(df.to_dict(orient='records')))
print(df.columns)

#########################
# Dashboard Layout / View
#########################
year = df['year'].unique() # filter item

image_filename = 'head_image.jpg'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(children=[
        html.B(html.H1('CDC COVID-19 Data')),
        html.Img(src=f'data:image/png;base64,{encoded_image.decode()}', style={'width': '200px'}),
    ]),
    html.Hr(),
    
    dcc.RadioItems(
        id='filter_type',
        options=[{'label': i, 'value': i} for i in year] + 
                 [{'label': 'Reset', 'value': 'Reset'}],
        value='Reset',
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
        style_header={'backgroundColor': 'lightblue', 'fontWeight': 'bold'},
        page_size=10
    ),
    
    html.Br(),
    html.Hr(),
    
    html.Div(className='row', style={'display' : 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update DataTable based on filter
@app.callback(Output('datatable-id', 'data'),
              [Input('filter_type', 'value')])
def update_dashboard(filter_type):
    if not filter_type or filter_type == 'Reset':
        dff = pd.DataFrame(list(db.read({})))  # All data
    else:
        dff = pd.DataFrame(list(db.filter_read(filter_type)))  # Filtered data

    if '_id' in dff:
        dff['_id'] = dff['_id'].apply(str)
        dff.drop(columns=['_id'], inplace=True)

    return dff.to_dict('records')

# Update Graph based on selected data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if not dff.empty:
        df_grouped = dff.groupby(['percent_of_expected_deaths']).size().reset_index(name='Count')
        fig = px.bar(df_grouped, 
                     x='percent_of_expected_deaths', 
                     y='Count', 
                     title="Count of Deaths",
                     labels={'percent_of_expected_deaths': 'Percent of Expected Deaths', 'Count': 'Number of Deaths'})
        return [dcc.Graph(figure=fig)]
    return []  # Return nothing if the data is empty

# Run app
if __name__ == '__main__':
    app.run(debug=True)  # Run in debug mode
